<a href="https://colab.research.google.com/github/abouslima/AI-Makerspace/blob/master/VertexAI/TorchGAN_Custom_Container.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:

PROJECT_ID="vertex-ai-makerspace"  # @param {type:"string"}
BUCKET_NAME="gs://gan-ai-makerspace-bucket1"  # @param {type:"string"}
REGION="europe-west4"  # @param {type:"string"}

In [10]:
gsutil mb -p $PROJECT_ID -l $REGION $BUCKET_NAME

Creating gs://gan-ai-makerspace-bucket1/...


In [3]:
mkdir gan

In [30]:
cd gan

In [ ]:
touch Dockerfile

**Dockerfile**

In [ ]:
FROM gcr.io/deeplearning-platform-release/pytorch-xla.1-10

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer
COPY requirements.txt requirements.txt

RUN pip install --no-cache-dir -r requirements.txt

# Sets up the entry point to invoke the trainer.
ENTRYPOINT ["python", "-m", "trainer.train"]

**requirements.txt**


In [ ]:
touch requirements.txt

In [ ]:
certifi==2021.10.8
charset-normalizer==2.0.12
docopt==0.6.2
idna==3.3
numpy==1.20
Pillow==9.0.1
pipreqs==0.4.11
requests==2.27.1
scipy==1.6.0
torch==1.10.2
torchgan==0.1.0
torchvision==0.11.3
typing_extensions==4.1.1
urllib3==1.26.8
wget==3.2
yarg==0.1.9

In [16]:
mkdir trainer

In [14]:
touch trainer/train.py

In [18]:
# General Imports
import random

# Pytorch and Torchvision Imports
import torch

from torch.optim import Adam
import torch.nn as nn
import torch.utils.data as data
import torchvision.datasets as dsets
import torchvision.transforms as transforms


# Torchgan Imports
from torchgan.models import *
from torchgan.losses import *
from torchgan.trainer import Trainer

# Set random seed for reproducibility
manualSeed = 999
random.seed(manualSeed)
torch.manual_seed(manualSeed)
print("Random Seed: ", manualSeed)


dataset = dsets.MNIST(
    root="./mnist",
    train=True,
    transform=transforms.Compose(
        [
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize(mean=(0.5,), std=(0.5,)),
        ]
    ),
    download=True,
)


dataloader = data.DataLoader(dataset, batch_size=64, shuffle=True)

dcgan_network = {
    "generator": {
        "name": DCGANGenerator,
        "args": {
            "encoding_dims": 100,
            "out_channels": 1,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.Tanh(),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0001, "betas": (0.5, 0.999)}},
    },
    "discriminator": {
        "name": DCGANDiscriminator,
        "args": {
            "in_channels": 1,
            "step_channels": 32,
            "nonlinearity": nn.LeakyReLU(0.2),
            "last_nonlinearity": nn.LeakyReLU(0.2),
        },
        "optimizer": {"name": Adam, "args": {"lr": 0.0003, "betas": (0.5, 0.999)}},
    },
}

minimax_losses = [MinimaxGeneratorLoss(), MinimaxDiscriminatorLoss()]
wgangp_losses = [
    WassersteinGeneratorLoss(),
    WassersteinDiscriminatorLoss(),
    WassersteinGradientPenalty(),
]
lsgan_losses = [LeastSquaresGeneratorLoss(), LeastSquaresDiscriminatorLoss()]


device = torch.device("cpu")
epochs = 10

print("Device: {}".format(device))
print("Epochs: {}".format(epochs))


trainer = Trainer(
    dcgan_network, lsgan_losses, sample_size=64, epochs=epochs, device=device
)

trainer(dataloader)

BUCKET_NAME="gs://gan-ai-makerspace-bucket1"
torch.save(trainer, BUCKET_NAME+"/gan/model")


Overwriting gan/trainer/train.py


In [19]:
IMAGE_URI="gcr.io/$PROJECT_ID/gan:v1"

In [21]:
IMAGE_URI

'gcr.io/vertex-ai-makerspace/gan:v1'

In [ ]:
docker build ./ -t $IMAGE_URI

In [ ]:
docker run $IMAGE_URI

In [ ]:
docker push $IMAGE_URI